In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
#Generate Cities List

In [3]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []


# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

609

In [4]:
cities

['mataura',
 'kahului',
 'urusha',
 'puerto ayora',
 'mathathane',
 'yellowknife',
 'raudeberg',
 'salalah',
 'rikitea',
 'upernavik',
 'saint-philippe',
 'praia',
 'san cristobal',
 'coruripe',
 'cape town',
 'butaritari',
 'pirapo',
 'tongzi',
 'awjilah',
 'codrington',
 'yamada',
 'mata',
 'naze',
 'esperance',
 'bam',
 'quime',
 'kaeo',
 'ushuaia',
 'eureka',
 'mandla',
 'kruisfontein',
 'kavieng',
 'barentsburg',
 'tuktoyaktuk',
 'faanui',
 'novobeysugskaya',
 'sumbuya',
 'nikolskoye',
 'geraldton',
 'khonuu',
 'dikson',
 'pitimbu',
 'ancud',
 'duderstadt',
 'teya',
 'bilibino',
 'orlik',
 'qaqortoq',
 'waipawa',
 'yar-sale',
 'punta arenas',
 'sao filipe',
 'kodiak',
 'boffa',
 'mezen',
 'marcona',
 'bluff',
 'barrow',
 'bonthe',
 'harsud',
 'havoysund',
 'albany',
 'jamestown',
 'zhanatas',
 'taolanaro',
 'bredasdorp',
 'vilyuysk',
 'san vicente de canete',
 'romans-sur-isere',
 'kutum',
 'atuona',
 'hobart',
 'samusu',
 'leningradskiy',
 'kapaa',
 'poronaysk',
 'hithadhoo',
 'c

In [5]:
# Perform API Calls

In [6]:
# Make portial query url for API call for 
url = f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}q="

In [7]:
# Make request and store response
# cities_data = requests.get(url).json()
# pprint(response.json())

In [8]:
# cities_data["main"]["sea_level"]

In [9]:
# Create empty list to hold required data
city_name = []

# initialize count variables for counting city and set
item = 1
batch = 1

# Print header before displaying for loop data
print("Beginning Data Retrieval")
print("----------------------------")

# for loop through list of cities and requstfor data on each city
for city in cities:
   
    # build query url with city
    q_url = url + city
    
    # get response as json
    cities_data = requests.get(url).json()
    
    # retrieve required data into list
    try:
        print(f"Processing Record {item} of Set {batch} | {city}")
        
        # increase item count by one
        item += 1
        
        # use if statemaent to limit API calls to 50
        if item > 50:
            
            # start a new set of items
            batch += 1
            # reset item count to 1 for next set
            item = 1
          
        #sleep time for 1 sec
            time.sleep(1)
    
    # if hit error        
    except:
        print(f"City '{city}' not found. Continue...")
        
# print completion header after displaing for loop data
print("----------------------------")
print("Data Retrieval Complete")
print("----------------------------")

Beginning Data Retrieval
----------------------------
Processing Record 1 of Set 1 | mataura
Processing Record 2 of Set 1 | kahului
Processing Record 3 of Set 1 | urusha
Processing Record 4 of Set 1 | puerto ayora
Processing Record 5 of Set 1 | mathathane
Processing Record 6 of Set 1 | yellowknife
Processing Record 7 of Set 1 | raudeberg
Processing Record 8 of Set 1 | salalah
Processing Record 9 of Set 1 | rikitea
Processing Record 10 of Set 1 | upernavik
Processing Record 11 of Set 1 | saint-philippe
Processing Record 12 of Set 1 | praia
Processing Record 13 of Set 1 | san cristobal
Processing Record 14 of Set 1 | coruripe
Processing Record 15 of Set 1 | cape town
Processing Record 16 of Set 1 | butaritari
Processing Record 17 of Set 1 | pirapo
Processing Record 18 of Set 1 | tongzi
Processing Record 19 of Set 1 | awjilah
Processing Record 20 of Set 1 | codrington
Processing Record 21 of Set 1 | yamada
Processing Record 22 of Set 1 | mata
Processing Record 23 of Set 1 | naze
Processin

Processing Record 50 of Set 4 | belaya gora
Processing Record 1 of Set 5 | karaton
Processing Record 2 of Set 5 | antsohihy
Processing Record 3 of Set 5 | bichura
Processing Record 4 of Set 5 | maningrida
Processing Record 5 of Set 5 | tiruvottiyur
Processing Record 6 of Set 5 | devonport
Processing Record 7 of Set 5 | mount gambier
Processing Record 8 of Set 5 | nanortalik
Processing Record 9 of Set 5 | anadyr
Processing Record 10 of Set 5 | portland
Processing Record 11 of Set 5 | ballina
Processing Record 12 of Set 5 | nizhneyansk
Processing Record 13 of Set 5 | bambari
Processing Record 14 of Set 5 | kirakira
Processing Record 15 of Set 5 | contai
Processing Record 16 of Set 5 | norman wells
Processing Record 17 of Set 5 | coihaique
Processing Record 18 of Set 5 | komsomolskiy
Processing Record 19 of Set 5 | rundu
Processing Record 20 of Set 5 | bidor
Processing Record 21 of Set 5 | changde
Processing Record 22 of Set 5 | kuytun
Processing Record 23 of Set 5 | marfino
Processing Re

Processing Record 1 of Set 9 | pombas
Processing Record 2 of Set 9 | mirskoy
Processing Record 3 of Set 9 | sabang
Processing Record 4 of Set 9 | borogontsy
Processing Record 5 of Set 9 | puerto quijarro
Processing Record 6 of Set 9 | marienburg
Processing Record 7 of Set 9 | eydhafushi
Processing Record 8 of Set 9 | nabire
Processing Record 9 of Set 9 | luderitz
Processing Record 10 of Set 9 | miramar
Processing Record 11 of Set 9 | marang
Processing Record 12 of Set 9 | conway
Processing Record 13 of Set 9 | abiy adi
Processing Record 14 of Set 9 | kuche
Processing Record 15 of Set 9 | tucupita
Processing Record 16 of Set 9 | sao joao da barra
Processing Record 17 of Set 9 | golden
Processing Record 18 of Set 9 | bandarbeyla
Processing Record 19 of Set 9 | olga
Processing Record 20 of Set 9 | cherskiy
Processing Record 21 of Set 9 | sumter
Processing Record 22 of Set 9 | murfreesboro
Processing Record 23 of Set 9 | ambon
Processing Record 24 of Set 9 | eldorado
Processing Record 25 o

Processing Record 49 of Set 12 | zadonsk
Processing Record 50 of Set 12 | amarillo
Processing Record 1 of Set 13 | tyrma
Processing Record 2 of Set 13 | ilheus
Processing Record 3 of Set 13 | okhotsk
Processing Record 4 of Set 13 | santa marta de tormes
Processing Record 5 of Set 13 | colares
Processing Record 6 of Set 13 | burnie
Processing Record 7 of Set 13 | hvolsvollur
Processing Record 8 of Set 13 | northampton
Processing Record 9 of Set 13 | vostok
----------------------------
Data Retrieval Complete
----------------------------


In [10]:
# Covert Raw Data to DataFrame
#--Export the city data into a csv
#--Display the DataFrame

In [11]:

# Define Dataframe
city_name = []
lat_range = []
lng_range = []
city_maxtemps = []
city_humidity = []
city_cloudiness = []
city_windspeed = []
city_country = []
date = []


#create a data frame 
city_temp_df = pd.DataFrame({"City": city_name,
                             "Lat": lat_range,
                             "Lng": lng_range,
                             "Max Temp": city_maxtemps,
                             "Humidity": city_humidity,
                             "Cloudiness": city_cloudiness,
                             "Wind Speed": city_windspeed, 
                             "Country": city_country,
                             "Date": date,
                             })

# save dataframe to .csv file
city_temp_df.to_csv("../output_data_file", index=True, index_label="City ID", header=True)

# display dataframe
city_temp_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
